# PyTorch Implementation of VGG-16 using Cifar-10 Dataset (Default)

## Loading Libaries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Pre-Processing

In [ ]:
# Hyper-parameters
num_epochs = 5
batch_size = 20
learning_rate = 0.001

### Transformation of ConvNet

In [ ]:
# Transformation for ConvNet
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

### Transformation of VGG-16

In [ ]:
# Transformation for VGG-16
transform = transforms.Compose([
                    transforms.Resize(size = (224,224)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])

### Loading Dataset

In [ ]:
# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
len(classes)

10

## Visualizing

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

AttributeError: ignored

In [ ]:
# show images
imshow(torchvision.utils.make_grid(images))

## Defining Model

### Setting Model Parameters

In [ ]:
# # Hyper-parameters
# num_epochs = 20
# learning_rate = 0.001

## Model Definition

### Normal ConvNet

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

### VGG-16


In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 32, 3, padding='same')
        self.batchnorm1_1 = nn.BatchNorm2d(32)
        self.conv1 = nn.Conv2d(32, 64, 3, padding='same')
        self.batchnorm1 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.5)


        self.conv2_1 = nn.Conv2d(64, 128, 3, padding='same')
        self.batchnorm2_1 = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(128, 128, 3, padding='same')
        self.batchnorm2 = nn.BatchNorm2d(128)

        self.conv3_1 = nn.Conv2d(128, 256, 3, padding='same')
        self.batchnorm3_1 = nn.BatchNorm2d(256)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding='same')
        self.batchnorm3_2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(256, 256, 3, padding='same')
        self.batchnorm3 = nn.BatchNorm2d(256)

        self.conv4_1 = nn.Conv2d(256, 512, 3, padding='same')
        self.batchnorm4_1 = nn.BatchNorm2d(512)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding='same')
        self.batchnorm4_2 = nn.BatchNorm2d(512)
        self.conv4 = nn.Conv2d(512, 512, 3, padding='same')
        self.batchnorm4 = nn.BatchNorm2d(512)

        # Reuse 3 times as in and out channel same:
        # We could just reuse the conv4 but conv5 is just a visual separation:
        self.conv5 = nn.Conv2d(512, 512, 3, padding='same')
        self.batchnorm5 = nn.BatchNorm2d(512)

        self.fc1 = nn.Linear(7*7*512, 4096)
        self.fc2 = nn.Linear(4096, 1000)
        self.fc3 = nn.Linear(1000, 10)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = self.batchnorm1_1(x)
        x = self.conv1(x)
        x = self.pool(F.relu(x))
        x = self.batchnorm1(x)


        x = F.relu(self.conv2_1(x))
        x = self.batchnorm2_1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.batchnorm2(x)

        x = F.relu(self.conv3_1(x))
        x = self.batchnorm3_1(x)
        x = F.relu(self.conv3_2(x))
        x = self.batchnorm3_2(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.batchnorm3(x)

        x = F.relu(self.conv4_1(x))
        x = self.batchnorm4_1(x)
        x = F.relu(self.conv4_2(x))
        x = self.batchnorm4_2(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = self.batchnorm4(x)


        x = F.relu(self.conv5(x))
        x = self.batchnorm5(x)
        x = F.relu(self.conv5(x))
        x = self.batchnorm5(x)
        x = self.pool(F.relu(self.conv5(x)))
        x = self.batchnorm5(x)

        x = x.view(-1, 7*7*512)
        # x = x.view(-1, 48*3*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

### Assigning Model

In [ ]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

### Training

In [ ]:
n_total_steps = len(train_loader)
print(n_total_steps)

running_loss = 0.0
running_correct = 0

2500


In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

KeyboardInterrupt: ignored

### Testing

#### ConvNet Test

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(len(n_class_correct)):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
        

Accuracy of the network: 60.49 %
Accuracy of plane: 67.2 %
Accuracy of car: 74.8 %
Accuracy of bird: 54.6 %
Accuracy of cat: 37.2 %
Accuracy of deer: 44.8 %
Accuracy of dog: 54.8 %
Accuracy of frog: 73.4 %
Accuracy of horse: 61.4 %
Accuracy of ship: 65.4 %
Accuracy of truck: 71.3 %


#### VGG-16 Test

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(len(n_class_correct)):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
        